## iPyLeaflet
[ipyleaflet](https://github.com/ellisonbg/ipyleaflet) is a bridge between jupyter notebooks and the [leaflet](http://leafletjs.com/)  javascript library for drawing maps.

ipyleaflet comes with a few examples notebooks (this notebook was derived from one) but very little documentation,
for more documentation read the [Leaflet IPA](http://leafletjs.com/reference.html)

For installation directions, see the README on [ipyleaflet](https://github.com/ellisonbg/ipyleaflet)

[GeoJson](https://tools.ietf.org/html/rfc7946#section-3.1.7) is a good way to add data on top of an ipyleaflet map.

For other backgrounds than open streets, you can alter the ipyleaflet.py file to point to one of [these map providers](https://leaflet-extras.github.io/leaflet-providers/preview/)

In [7]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

meas = 'TOBS'

## Visualizing the distribution of the observations

## Load the required libraries

In [8]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
import pylab as plt

import sys
sys.path.append('./lib')

from leaflet import *

In [9]:
data_dir = "../../Data/Weather/"
from pickle import load
with open(data_dir+meas+'_residuals_PCA.pickle','rb') as file:
    Res_eig=load(file)
print Res_eig.keys()

#stations.pkl
with open(data_dir+'stations.pkl','rb') as file:
    Stations=load(file)
Stations.head()


['eigen-vecs', 'stations']


latitude  longitude  elevation state                   name  \
ACW00011604   17.1167   -61.7833       10.1   NaN  ST JOHNS COOLIDGE FLD   
ACW00011647   17.1333   -61.7833       19.2   NaN               ST JOHNS   
AE000041196   25.3330    55.5170       34.0   NaN    SHARJAH INTER. AIRP   
AF000040930   35.3170    69.0170     3366.0   NaN           NORTH-SALANG   
AG000060390   36.7167     3.2500       24.0   NaN     ALGER-DAR EL BEIDA   

            GSNFLAG HCNFLAG    WMOID  
ACW00011604     NaN     NaN      NaN  
ACW00011647     NaN     NaN      NaN  
AE000041196     GSN     NaN  41196.0  
AF000040930     GSN     NaN  40930.0  
AG000060390     GSN     NaN  60390.0

In [10]:
import pandas as pd
Eig=pd.DataFrame(Res_eig['eigen-vecs'].transpose())
Eig.head()

0         1         2         3         4         5         6  \
0 -0.073361 -0.307084 -0.123486  0.122552  0.238617 -0.001687 -0.049322   
1 -0.027402 -0.287962 -0.031383 -0.269599  0.319116 -0.379029 -0.137349   
2 -0.056049 -0.229595 -0.057984  0.192101  0.212466  0.134252  0.021623   
3 -0.053147 -0.136714 -0.155152 -0.075409  0.085110  0.141535 -0.005137   
4 -0.222955  0.056242 -0.011120 -0.180364  0.256125 -0.214774 -0.131503   

          7         8         9  
0  0.036204 -0.111491  0.032996  
1  0.271419 -0.044260  0.147431  
2 -0.036835 -0.054078  0.021439  
3  0.123719 -0.014987 -0.299716  
4 -0.034093  0.134640  0.055129

In [11]:
Eig['station']=Res_eig['stations']
Eig=Eig.set_index('station')
print Res_eig['stations'][:5]
Eig.head()

[u'USC00094238', u'USC00098351', u'USW00093845', u'USC00094255', u'USC00094429']


0         1         2         3         4         5  \
station                                                                   
USC00094238 -0.073361 -0.307084 -0.123486  0.122552  0.238617 -0.001687   
USC00098351 -0.027402 -0.287962 -0.031383 -0.269599  0.319116 -0.379029   
USW00093845 -0.056049 -0.229595 -0.057984  0.192101  0.212466  0.134252   
USC00094255 -0.053147 -0.136714 -0.155152 -0.075409  0.085110  0.141535   
USC00094429 -0.222955  0.056242 -0.011120 -0.180364  0.256125 -0.214774   

                    6         7         8         9  
station                                              
USC00094238 -0.049322  0.036204 -0.111491  0.032996  
USC00098351 -0.137349  0.271419 -0.044260  0.147431  
USW00093845  0.021623 -0.036835 -0.054078  0.021439  
USC00094255 -0.005137  0.123719 -0.014987 -0.299716  
USC00094429 -0.131503 -0.034093  0.134640  0.055129

In [12]:
table=Eig.join(Stations,how='left')

In [13]:
table=table[['name','latitude','longitude','elevation',0,1,2,3,4,5,6,7]]
table.head(4)

name  latitude  longitude  elevation         0  \
station                                                                   
USC00094238            HELENA   32.0667   -82.9167       76.2 -0.073361   
USC00098351         STILLMORE   32.4414   -82.2181       79.2 -0.027402   
USW00093845  VALDOSTA RGNL AP   30.7825   -83.2767       60.4 -0.056049   
USC00094255         HEPHZIBAH   33.3167   -82.0833      121.9 -0.053147   

                    1         2         3         4         5         6  \
station                                                                   
USC00094238 -0.307084 -0.123486  0.122552  0.238617 -0.001687 -0.049322   
USC00098351 -0.287962 -0.031383 -0.269599  0.319116 -0.379029 -0.137349   
USW00093845 -0.229595 -0.057984  0.192101  0.212466  0.134252  0.021623   
USC00094255 -0.136714 -0.155152 -0.075409  0.085110  0.141535 -0.005137   

                    7  
station                
USC00094238  0.036204  
USC00098351  0.271419  
USW00093845 -0.036835  
USC00094255  0.123719

In [14]:
import pylab as plt
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

## Map

In [16]:
#min_lat,max_lat,min_long,max_long = box = (42.1103, 42.6167, -72.6, -70.8)
min_lat,max_lat,min_long,max_long = box = (30, 34, -84, -82)
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9
m = Map(default_tiles=TileLayer(opacity=0.6), center=center, zoom=zoom)
#r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
for index,row in table.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    for col in range(4):
        _coef=row[col]
        if np.isnan(_coef):
            continue
        r=abs(_coef)/5
        #print r
        color=colors[col]
        signs=[[+1,+1],[+1,-1],[-1,-1],[-1,+1]]
        lat_sign,long_sign=signs[col]
        triangle=[(_lat,_long),(_lat+lat_sign*r,_long),(_lat,_long+long_sign*r),(_lat,_long)]
        if _coef<0:
            poly= Polygon(locations=triangle, weight=0,
                    color=color, opacity=0, fill_opacity=0.7,
                    fill_color=color)
        else:
            poly= Polygon(locations=triangle, weight=2,
                    color=color, opacity=0.8, fill_opacity=0,
                    fill_color=color)
        m +=poly   
m    

### excercises:
* Add a legend that relates the colors to values.
* Leaflet supports a variety of maps. See if you can get a topographical map as the background.

In [19]:
%matplotlib inline
pdf.plot.scatter(x='elevation',y='avg(coeff_1)');

NameError: name 'pdf' is not defined